In [2]:
import pandas as pd

df = pd.read_csv("fisheries_with_pace_kd.csv")
print(df.columns)

Index(['TOWDATETIME_EST', 'LAT', 'LON', 'MEAN_DEPTH', 'SWEPT_AREA_km',
       'acadian redfish', 'alewife', 'alligatorfish', 'american lobster',
       'american plaice',
       ...
       'Kd_510', 'Kd_532', 'Kd_555', 'Kd_583', 'Kd_618', 'Kd_640', 'Kd_655',
       'Kd_665', 'Kd_678', 'Kd_711'],
      dtype='object', length=217)


In [3]:
import pandas as pd
import numpy as np


# Step 1: Define metadata and Rrs columns
metadata_cols = ['TOWDATETIME_EST', 'LAT', 'LON', 'MEAN_DEPTH', 'SWEPT_AREA_KM']

# Step 2: Define spectral Kd columns
kd_cols = [col for col in df.columns if col.startswith("Kd_")]

# Step 3: Define species columns (everything else)
species_cols = [col for col in df.columns if col not in metadata_cols + kd_cols]

# Step 4: Convert to numeric
df[species_cols + kd_cols] = df[species_cols + kd_cols].apply(pd.to_numeric, errors='coerce')

# === Load surface Es spectrum ===
df_es = pd.read_csv('Es_spectrum.csv')  # should have 'wave' and 'Es' columns
wave_Es = df_es['wave'].values  # e.g., [412, 443, ...]
Es = df_es['Es'].values         # same length as kd_cols


In [4]:
# Prepare output
attenuated_Es_list = []

for i, row in df.iterrows():
    z = row["MEAN_DEPTH"]
    Kd = row[kd_cols].values.astype(float)  # shape = [n_wavelengths]
    
    # Skip if any Kd is missing
    if np.isnan(Kd).any() or pd.isna(z):
        attenuated_Es_list.append([np.nan] * len(Kd))  # keep row structure
        continue

    # Element-wise attenuation: Es * exp(-Kd * z)
    Edz = Es * np.exp(-Kd * z)
    attenuated_Es_list.append(Edz)

# Label columns: match Es wavelengths
edz_cols = [f"Edz_{int(w)}" for w in wave_Es]

# Create new DataFrame
df_Edz = pd.DataFrame(attenuated_Es_list, columns=edz_cols)

# Join to main DataFrame
df = pd.concat([df, df_Edz], axis=1)

df.to_csv("fisheries_with_pace_kd_Edz.csv", index=False)

In [16]:
df_with_Edz = pd.read_csv("fisheries_with_pace_kd_Edz.csv")

# Extract only the Edz columns (assuming they have the form 'Edz_XXX')
edz_cols = [col for col in df_with_Edz.columns if col.startswith("Edz_")]

# Extract the Edz array
edz_array = df_with_Edz[edz_cols].values

# Wavelengths from your Es spectrum (adjust if necessary)
wave_Es = df_es['wave'].values

# Trapezoidal integration
df_with_Edz["Edz_trapz"] = (np.trapz(edz_array, wave_Es, axis=1))/3.610104
df_with_Edz["Edz_trapz_log"] = np.log10(np.trapz(edz_array, wave_Es, axis=1))

# Apparent Visible Wavelength (AVW)
with np.errstate(divide='ignore', invalid='ignore'):
    numerator = np.sum(edz_array, axis=1)
    denominator = np.sum(edz_array / wave_Es, axis=1)
    df_with_Edz["AVW_Edz"] = numerator / denominator

df_with_Edz.to_csv("fisheries_with_pace_kd_Edz_AVW_PAR.csv", index=False)

/tmp/ipykernel_3884/900112321.py:13: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df_with_Edz["Edz_trapz"] = (np.trapz(edz_array, wave_Es, axis=1))/3.610104
/tmp/ipykernel_3884/900112321.py:14: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df_with_Edz["Edz_trapz_log"] = np.log10(np.trapz(edz_array, wave_Es, axis=1))
